# Sweep pytorch tutorial

📢 해당 노트북은 Wandb 공식 홈페이지 중 다음 내용을 참고했습니다. 
  
[Organizing Hyperparameter Sweeps in PyTorch with W&B](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch/Organizing_Hyperparameter_Sweeps_in_PyTorch_with_W%26B.ipynb#scrollTo=6IgMNFFT2qe_)  
[Sweep from Jupyter Notebook](https://docs.wandb.ai/v/ko/sweeps/python-api)  


### 목차  
Step 0: W&B Setup하기  
Step 1: Congfig 정의하기  
Step 2: Dataloader 함수 정의하기  
Step 3: 모델 정의하기  
Step 4: Train 함수와 Vaild 함수 정의하기  
Step 5: Sweep 실행하기  

## Step 0: W&B Setup하기
colab에서 WandB를 사용하려면 wandb modul을 install 해야합니다.

In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import wandb
import math
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
%%capture
!pip install wandb --upgrade

In [3]:
import wandb

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: asdqwdasdqwdzxcxzv (use `wandb login --relogin` to force relogin)


True

## Step1. Config 정의하기

sweep에 필요한 config 파일을 정의합니다.

In [38]:
sweep_config = {
    'name' : 'bayes-test',
    'method': 'random',
    'metric' : {
        'name': 'val_loss',
        'goal': 'minimize'   
        },
    'parameters' : {
        'optimizer': {
            'values': ['adam', 'sgd']
            },
        'dropout': {
            'values': [0.3, 0.4]
            },
        'learning_rate': {
            'distribution': 'uniform',
            'min': 0,
            'max': 0.1
            },
        'epochs': {
            'values': [5, 6]
            },
        'batch_size': {
            'distribution': 'q_log_uniform',
            'q': 1,
            'min': math.log(32),
            'max': math.log(256),
            }
        }
    }

## Step2. Dataloader 함수 정의하기
mnist 함수를 사용해서 dataloader를 구성하겠습니다.

In [39]:
def SweepDataset(batch_size):
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307,), (0.3081,))])
    
    train_data = datasets.MNIST(".", 
                train=True, 
                download=True,
                transform=transform)
    
    test_data = datasets.MNIST(".", 
                train=False, 
                download=True,
                transform=transform)
    
    
    train_loader = DataLoader(train_data, batch_size=batch_size)
    test_loader = DataLoader(test_data, batch_size=batch_size)

    return train_loader, test_loader

## Step3. 모델 정의하기
모델과 optimizer를 정의해줍니다.

In [40]:
class ConvNet(nn.Module):
    def __init__(self, dropout):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1, 1), nn.ReLU(),
            nn.MaxPool2d(2, 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, 1, 1), nn.ReLU(),
            nn.MaxPool2d(2, 2))
        self.layer3 = nn.Sequential(
            nn.Linear(64 * 7 * 7, 128, bias=True), nn.ReLU(),
            nn.Dropout2d(p=dropout))
        self.layer4 = nn.Sequential(
            nn.Linear(128, 84), nn.ReLU(),
            nn.Dropout2d(p=dropout))
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0),-1) 
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.fc3(x)
        return x

In [41]:
def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer

## Step4. Train 함수와 Vaild함수 정의하기
train함수는 학습을 진행하면서 loss를 출력합니다.  
vaild함수는 학습 중간에 Accuracy를 사용해서 평가합니다.

In [48]:
def train(model, loader, criterion, optimizer, device, config, wandb):
    model.train()
    wandb.watch(model, criterion, log="all", log_freq=10)

    for epoch in range(config.epochs):
        cumu_loss = 0
        for images, labels in loader:
            images, labels  = images.to(device), labels.to(device)

            output = model(images)
            loss = criterion(output, labels)
            cumu_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        avg_loss = cumu_loss / len(loader)
        wandb.log({"train_loss": avg_loss}, step=epoch)  
        print(f"TRAIN: EPOCH {epoch + 1:04d} / {config.epochs:04d} | Epoch LOSS {avg_loss:.4f}")

In [49]:
def vaild(model, loader, criterion, device,  wandb):
    model.eval()

    with torch.no_grad():
        correct, test_loss = 0, 0
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            test_loss += criterion(output, target).item()
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    
    val_loss = test_loss / len(loader)
    print(f"VALID: LOSS {val_loss:.4f} | Accuracy {val_loss:.4f} ")
    wandb.log({
        "val_acc": 100. * correct / len(loader.dataset),
        "val_loss": val_loss})

## Step5. Sweep 실행하기

In [50]:
def run_sweeep(config=None):
    wandb.init(config=config)
    
    w_config = wandb.config
    
    criterion = nn.CrossEntropyLoss()
    train_loader, vaild_loader = SweepDataset(w_config.batch_size)
    model = ConvNet(w_config.dropout).to(DEVICE)
    optimizer = build_optimizer(model, w_config.optimizer, w_config.learning_rate)
    
    train(model, train_loader, criterion, optimizer, DEVICE, w_config, wandb)
    vaild(model, vaild_loader, criterion, DEVICE, wandb)

In [51]:
sweep_id = wandb.sweep(sweep_config, project="sweep_tutorial", entity='pebpung')
wandb.agent(sweep_id, run_sweeep, count=6)

Create sweep with ID: 8mruz4pk
Sweep URL: https://wandb.ai/pebpung/sweep_tutorial/sweeps/8mruz4pk


wandb: Agent Starting Run: wdakaxg2 with config:
wandb: 	batch_size: 224
wandb: 	dropout: 0.3
wandb: 	epochs: 5
wandb: 	learning_rate: 0.0791329183434094
wandb: 	optimizer: adam


TRAIN: EPOCH 0001 / 0005 | Epoch LOSS 6.6979
TRAIN: EPOCH 0002 / 0005 | Epoch LOSS 2.3035
TRAIN: EPOCH 0003 / 0005 | Epoch LOSS 2.3036
TRAIN: EPOCH 0004 / 0005 | Epoch LOSS 2.3036
TRAIN: EPOCH 0005 / 0005 | Epoch LOSS 2.3036
VALID: LOSS 2.3016 | Accuracy 2.3016 


train_loss,█▁▁▁▁
val_acc,▁
val_loss,▁
train_loss,2.30363
val_acc,11.35
val_loss,2.30157


wandb: Agent Starting Run: ulezb7kv with config:
wandb: 	batch_size: 159
wandb: 	dropout: 0.4
wandb: 	epochs: 5
wandb: 	learning_rate: 0.00936485889539045
wandb: 	optimizer: sgd


...

TRAIN: EPOCH 0001 / 0005 | Epoch LOSS 2.9021
TRAIN: EPOCH 0002 / 0005 | Epoch LOSS 2.3035
TRAIN: EPOCH 0003 / 0005 | Epoch LOSS 2.3035
TRAIN: EPOCH 0004 / 0005 | Epoch LOSS 2.3036
TRAIN: EPOCH 0005 / 0005 | Epoch LOSS 2.3036
VALID: LOSS 2.3021 | Accuracy 2.3021 


train_loss,█▁▁▁▁
val_acc,▁
val_loss,▁
train_loss,2.30357
val_acc,10.1
val_loss,2.30207


wandb: Agent Starting Run: qw10xfkb with config:
wandb: 	batch_size: 68
wandb: 	dropout: 0.3
wandb: 	epochs: 5
wandb: 	learning_rate: 0.021868136115833214
wandb: 	optimizer: adam


TRAIN: EPOCH 0001 / 0005 | Epoch LOSS 2.3277
TRAIN: EPOCH 0002 / 0005 | Epoch LOSS 2.3025
TRAIN: EPOCH 0003 / 0005 | Epoch LOSS 2.3025
TRAIN: EPOCH 0004 / 0005 | Epoch LOSS 2.3025
TRAIN: EPOCH 0005 / 0005 | Epoch LOSS 2.3025
VALID: LOSS 2.3016 | Accuracy 2.3016 


train_loss,█▁▁▁▁
val_acc,▁
val_loss,▁
train_loss,2.30254
val_acc,11.35
val_loss,2.30159
